In [4]:
from google.colab import drive
drive.mount('/content/drive')

! pip install bitsandbytes
! pip install huggingface_hub
! pip install transformers
! pip install torch
! pip install accelerate
! pip install unsloth

# OR
# ! pip install -r requirements.txt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import torch
from unsloth import FastLanguageModel
import os
import re

MAX_SEQUENCE_LENGTH = 8192
CHAPTER_TOKEN_LIMIT = 8192
MODEL_NAME = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"

# Define output directory for the model
output_dir = "/content/drive/MyDrive/Colab Notebooks/Novel-Writer/output"
os.makedirs(output_dir, exist_ok=True)

# Define knowledge data base path
knowledge_db_dir = "/content/drive/MyDrive/Colab Notebooks/Novel-Writer/knowledge_db"

# Set HuggingFace cache directory
os.makedirs("/content/drive/MyDrive/Colab Notebooks/Novel-Writer/output", exist_ok=True)
os.environ["HF_HOME"] = "/content/drive/MyDrive/Colab Notebooks/Novel-Writer/output"
print(f"Using HuggingFace cache directory: {os.environ['HF_HOME']}")

# Load the FastLanguageModel with 4-bit quantization
print(f"Loading model {MODEL_NAME} with 4-bit quantization...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQUENCE_LENGTH,
    dtype=torch.bfloat16,  # Use bfloat16 for better precision if your GPU supports it
    load_in_4bit=True,  # Load in 4-bit quantization for memory efficiency
)

Using HuggingFace cache directory: /content/drive/MyDrive/Colab Notebooks/Novel-Writer/output
Loading model unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit with 4-bit quantization...
==((====))==  Unsloth 2025.6.2: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Device does not support bfloat16. Will change to float16.


In [9]:
# --- Step 1: Extract the specific chapter prompt for Chapter 1 ---
# This section has been modified to use your refined logic for robustness!
chapter_prompts_file_path = os.path.join(knowledge_db_dir, "rwby_chapter_prompts.md")
chapter_number = 1 # We are generating Chapter 1

print(f"Extracting prompt for Chapter {chapter_number} from {chapter_prompts_file_path}...")

# Read all the contents of the prompt file
try:
    with open(chapter_prompts_file_path, "r") as file:
        all_prompts = file.read()
except FileNotFoundError:
    print(f"Error: The chapter prompts file '{chapter_prompts_file_path}' was not found.")
    print("Please ensure your Google Drive is mounted correctly and the file exists at the specified path.")
    exit() # Exit if the file is not found

# Define the starting and ending markers based on our prompt markdown structure
starting_phrase = f"## Chapter {chapter_number} Prompt:"
ending_phrase = f"## Chapter {chapter_number + 1} Prompt:" # Look for the next chapter's heading

# Find the start and end indices of the desired chapter's prompt section
start_index = all_prompts.find(starting_phrase)
end_index = all_prompts.find(ending_phrase, start_index)

# Handle the case if the next chapter's prompt isn't found (e.g., if it's the last chapter)
if end_index == -1:
    # If it's the last chapter, read until the end of the file
    chapter_section_raw = all_prompts[start_index:].strip()
else:
    chapter_section_raw = all_prompts[start_index:end_index].strip()

# Error handling if the starting prompt for the chapter is not found
if start_index == -1:
    raise ValueError(f"Could not find the starting phrase '{starting_phrase}' for Chapter {chapter_number} in the prompts file.")

# Now, extract the actual prompt text, removing the heading and markdown code block.
# We're looking for the '```\n' marker which indicates the start of the actual prompt content.
prompt_content_start_marker = "```\n"
content_start_index = chapter_section_raw.find(prompt_content_start_marker)

chapter_prompt_text = ""
if content_start_index != -1:
    # Extract everything after the '```\n' marker
    chapter_prompt_text = chapter_section_raw[content_start_index + len(prompt_content_start_marker):].strip()
    # Remove the closing '```' if it exists at the end
    if chapter_prompt_text.endswith("```"):
        chapter_prompt_text = chapter_prompt_text[:-3].strip()
else:
    # Fallback: if no code block marker, assume prompt starts after the heading and two newlines.
    end_of_heading_index = chapter_section_raw.find("\n\n")
    if end_of_heading_index != -1:
        chapter_prompt_text = chapter_section_raw[end_of_heading_index + 2:].strip()
    print(f"Warning: No '```' found for Chapter {chapter_number} prompt. Using fallback parsing.")

# The final 'chapter_prompt' variable holds the clean, extracted text
chapter_prompt = chapter_prompt_text
print("Chapter prompt extracted successfully!")
# print(chapter_prompt) # Uncomment this to verify the extracted prompt

# --- Step 2: Read the contents of the *other* Knowledge Database files ---
print("Reading supporting Knowledge Database files...")
characters_content = ""
locations_content = ""
plot_events_content = ""

try:
    with open(os.path.join(knowledge_db_dir, "rwby_characters.md"), "r") as f:
        characters_content = f.read()
    with open(os.path.join(knowledge_db_dir, "rwby_locations.md"), "r") as f:
        locations_content = f.read()
    with open(os.path.join(knowledge_db_dir, "rwby_plot_events.md"), "r") as f:
        plot_events_content = f.read()
    print("Supporting Knowledge Database files loaded successfully!")
except FileNotFoundError as e:
    print(f"Error: A supporting Knowledge Database file was not found. Please ensure all files are correctly mounted/uploaded. {e}")
    exit() # Exit if supporting files are missing


# --- Step 3: Extract relevant sections for Chapter 1 from supporting files ---
# Characters for Chapter 1: Ruby, Weiss, Blake, Yang, Jaune
relevant_characters_text = ""
character_names_for_chapter = ["Ruby Rose", "Weiss Schnee", "Blake Belladonna", "Yang Xiao Long", "Jaune Arc"]
for name in character_names_for_chapter:
    start_marker = f"## {name}\n"
    start_index = characters_content.find(start_marker)
    if start_index != -1:
        # Find the end of the character's section (start of next character or end of file)
        end_index = characters_content.find("## ", start_index + len(start_marker))
        if end_index == -1: # If it's the last character in the file
            end_index = len(characters_content)
        relevant_characters_text += characters_content[start_index:end_index].strip() + "\n\n"
    else:
        print(f"Warning: Character '{name}' not found in rwby_characters.md. Skipping for prompt.")

# Plot Events for Chapter 1: Volume 9 (most recent context)
volume_9_start_marker = "## Volume 9: The Ever After Arc & Return\n"
volume_9_start_index = plot_events_content.find(volume_9_start_marker)
relevant_plot_events_text = ""
if volume_9_start_index != -1:
    # Since Volume 9 is the last, read till end of file for its details
    relevant_plot_events_text = plot_events_content[volume_9_start_index:].strip()
else:
    print("Warning: Volume 9 plot events not found in rwby_plot_events.md. Skipping for prompt.")

# Locations for Chapter 1: Vacuo
vacuo_start_marker = "## Vacuo\n"
vacuo_start_index = locations_content.find(vacuo_start_marker)
relevant_locations_text = ""
if vacuo_start_index != -1:
    # Find the end of Vacuo section (start of next location or end of file)
    vacuo_end_index = locations_content.find("## ", vacuo_start_index + len(vacuo_start_marker))
    if vacuo_end_index == -1:
        vacuo_end_index = len(locations_content)
    relevant_locations_text = locations_content[vacuo_start_index:vacuo_end_index].strip()
else:
    print("Warning: Vacuo location details not found in rwby_locations.md. Skipping for prompt.")


# --- Step 4: Construct the Augmented Prompt ---
augmented_prompt_content = f"""Here is relevant background information from the RWBY universe to inform your writing. This is crucial for maintaining narrative consistency and character integrity.

---BEGIN CHARACTERS DATABASE---
{relevant_characters_text}
---END CHARACTERS DATABASE---

---BEGIN LOCATIONS DATABASE---
{relevant_locations_text}
---END LOCATIONS DATABASE---

---BEGIN PLOT HISTORY DATABASE (Volume 9 is most recent context)---
{relevant_plot_events_text}
---END PLOT HISTORY DATABASE---

Now, please follow the specific instructions for generating Chapter {chapter_number}:
{chapter_prompt}
"""

# --- Step 5: Prepare the prompt for the model's chat template ---
messages = [
    {"role": "user", "content": augmented_prompt_content},
]
prompt_formatted = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print("\n--- Augmented Prompt Prepared ---")
# print(prompt_formatted) # Uncomment this to see the full prompt being sent to the model


# --- Step 6: Generate the Chapter! ---
print(f"Generating Chapter {chapter_number}... This may take a moment!")
# Tokenize the prompt and move tensors to GPU
tokenized_inputs = tokenizer(prompt_formatted, return_tensors="pt")
inputs = {k: v.to("cuda") for k, v in tokenized_inputs.items()} # Move both input_ids and attention_mask

# Generate the text
outputs = model.generate(
    input_ids = inputs['input_ids'],
    attention_mask = inputs['attention_mask'],
    max_new_tokens = CHAPTER_TOKEN_LIMIT,
    use_cache = True,
    temperature = 0.8,
    top_p = 0.9,
    do_sample = True,
    pad_token_id = tokenizer.eos_token_id,
)
print(f"Chapter {chapter_number} generation complete!")

# --- Step 7: Decode and Clean the Generated Text ---
# Decode the *entire* output sequence first, without skipping special tokens initially,
# so we can parse the chat template structure.
full_response_with_template = tokenizer.decode(outputs[0], skip_special_tokens=False)

# --- Crucial Cleaning Logic ---
# Find the marker that signals the start of the assistant's *actual response*.
# For Llama-3.1 models with apply_chat_template, this is typically the end of the assistant header.
# The template for a user-assistant turn is often:
# <s><|start_header_id|>user<|end_header_id|>
# [USER CONTENT]
# <|eot_id|><|start_header_id|>assistant<|end_header_id|>
# [ASSISTANT RESPONSE]
# <|eot_id|>

# We are looking for the point right after the assistant's header.
assistant_response_start_marker = "<|start_header_id|>assistant<|end_header_id|>\n\n"
start_of_generation_index = full_response_with_template.find(assistant_response_start_marker)

chapter_text = ""
if start_of_generation_index != -1:
    # Slice from *after* the assistant's marker
    raw_generated_content = full_response_with_template[start_of_generation_index + len(assistant_response_start_marker):]

    # Then, remove any remaining special tokens (like <|eot_id|>) and excessive whitespace
    chapter_text = tokenizer.decode(tokenizer.encode(raw_generated_content), skip_special_tokens=True).strip()
else:
    # Fallback if the template marker isn't found (less likely now)
    chapter_text = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    print("Warning: Assistant response start marker not found. Using fallback decoding.")

# Remove any potential leading unwanted dialogue roles or system messages that might still slip through
# This acts as a final safeguard after the slicing.
unwanted_prefixes = [
    "system",
    "user",
    "assistant",
    "Here is relevant background information from the RWBY universe to inform your writing.",
    "---BEGIN CHARACTERS DATABASE---",
    "---END CHARACTERS DATABASE---",
    "---BEGIN LOCATIONS DATABASE---",
    "---END LOCATIONS DATABASE---",
    "---BEGIN PLOT HISTORY DATABASE (Volume 9 is most recent context)---",
    "---END PLOT HISTORY DATABASE---",
    "Now, please follow the specific instructions for generating Chapter 1:",
    "Chapter 1 generated and saved to:",
    "--- First Chapter Ready! ---",
    "cutting knowledge date:", # Case insensitive check
    "today date:",           # Case insensitive check
]

# Convert chapter_text to lowercase for initial prefix matching for robustness
temp_chapter_text_lower = chapter_text.lower()
for prefix in unwanted_prefixes:
    lower_prefix = prefix.lower().replace("...", "") # Remove ellipses for matching
    while temp_chapter_text_lower.startswith(lower_prefix):
        # Find the length of the actual prefix in the original case text
        original_prefix_len = len(prefix) # Approximate or find actual if complex
        if chapter_text.lower().startswith(lower_prefix): # Confirm match in original case
             # Find first space/newline after prefix to cut cleanly
            cut_point = len(lower_prefix) # default cut point
            match = next((m for m in re.finditer(re.escape(prefix), chapter_text, re.IGNORECASE) if m.start() == 0), None)
            if match:
                cut_point = match.end()
                # Find end of line or next paragraph if it's a heading-like prefix
                next_newline = chapter_text.find('\n', cut_point)
                if next_newline != -1:
                    cut_point = next_newline + 1
                    next_newline2 = chapter_text.find('\n', cut_point)
                    if next_newline2 != -1 and chapter_text[cut_point:next_newline2].strip() == "": # Skip empty line
                        cut_point = next_newline2 + 1

            chapter_text = chapter_text[cut_point:].strip()
            temp_chapter_text_lower = chapter_text.lower() # Update for next iteration
        else:
            break # No match, stop trying this prefix

# Final strip just in case
chapter_text = chapter_text.strip()

# --- Step 8: Save the Generated Chapter ---
chapter_filename = os.path.join(output_dir, f"chapter_{chapter_number:02d}.md") # Format as chapter_01.md
with open(chapter_filename, "w") as f:
    f.write(chapter_text)

print(f"\nChapter {chapter_number} generated and saved to: {chapter_filename}")
print(f"\n--- Chapter {chapter_number} Ready! The story has begun! ---")

# Optional: Print the first few lines of the generated chapter to see it
print(f"\n--- First few lines of Chapter {chapter_number} ---")
print(chapter_text[:1000]) # Print first 1000 characters

Extracting prompt for Chapter 1 from /content/drive/MyDrive/Colab Notebooks/Novel-Writer/knowledge_db/rwby_chapter_prompts.md...
Chapter prompt extracted successfully!
Reading supporting Knowledge Database files...
Supporting Knowledge Database files loaded successfully!

--- Augmented Prompt Prepared ---
Generating Chapter 1... This may take a moment!
Chapter 1 generation complete!

Chapter 1 generated and saved to: /content/drive/MyDrive/Colab Notebooks/Novel-Writer/output/chapter_01.md

--- Chapter 1 Ready! The story has begun! ---

--- First few lines of Chapter 1 ---
Chapter 1: Dust and Desert

As the swirling vortex dissipated, the group stumbled out of the shimmering portal, their bodies protesting the sudden return to Remnant's harsh environment. Ruby's eyes watered from the blinding sunlight, her gaze drifting over the endless dunes of Vacuo's desert. She blinked, taking in the desolate landscape – a stark contrast to the vibrant, fairytale-inspired world they'd left behind.

